# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

To answer question one, we can create a total_sum variable with a value of zero, and then using the for loop to add the results of the generated numbers.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

total_sum = 0
for sqrt_value in generator:
    total_sum += sqrt_value
print(total_sum)


8.382332347441762


To answer question two and find the 13th number yielded by the square_root_generator function, you would set the limit to a number equal to or greater than 13, since you want to ensure the generator yields at least 13 values. From there, you can iterate through the generator and count until you reach the 13th value.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 13  # Ensure the limit is at least 13
generator = square_root_generator(limit)

# Initialize a variable to keep track of the count
count = 0
thirteenth_value = None

# Iterate through the generator
for value in generator:
    count += 1
    if count == 13:
        thirteenth_value = value
        break  # Exit the loop once the 13th value is found

print(f"The 13th number yielded is: {thirteenth_value}")

The 13th number yielded is: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [4]:
import dlt

#Creating the pipeline with dlt library

generator_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

#Loading the first generator to the data base

info = generator_pipeline.run(people_1(),
                                 table_name="people_1",
                                 write_disposition="replace")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/Frank/Documentos/data_eng_zoom/workshop_dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707606705.683861 is LOADED and contains no failed jobs


In [8]:
import duckdb

#Connecting to the duckdb

conn = duckdb.connect(f"{generator_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generator_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n people_1 table below:")

sql_people_1 = conn.sql("SELECT * FROM people_1").df()
display(sql_people_1)

print("\n\n\n Sum of ages of all people in the first generator")

#Printing the sum ages of all people in the first data

age_people_1 = conn.sql("SELECT SUM(AGE) FROM people_1").df()
display(age_people_1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
│ people_1            │
│ stream_download     │
└─────────────────────┘




 people_1 table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707606705.683861,umMUkts+G5fbLg
1,2,Person_2,27,City_A,1707606705.683861,+SnlsDYp8DUQZA
2,3,Person_3,28,City_A,1707606705.683861,yTnturJajAzODQ
3,4,Person_4,29,City_A,1707606705.683861,ziMlGJuq+dvbhg
4,5,Person_5,30,City_A,1707606705.683861,2qWIfWT9AfNDQQ





 Sum of ages of all people in the first generator


,sum(AGE)
0,140.0


2. Append the second generator to the same table as the first.

In [9]:
info = generator_pipeline.run(people_2(),
                                 table_name="people_1",
                                 write_disposition="append")

3. **After correctly appending the data, calculate the sum of all ages of people.**

In [12]:
age_all_people = conn.sql("SELECT SUM(AGE) FROM people_1").df()
display(age_all_people)

,sum(AGE)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [13]:
generator_pipeline_2 = dlt.pipeline(destination='duckdb', dataset_name='generators')

info_2 = generator_pipeline_2.run(people_1(),
                                 table_name="people_merge",
                                 write_disposition="replace")

info_2 = generator_pipeline_2.run(people_2(),
                                 table_name="people_merge",
                                 write_disposition="merge",
                                 primary_key="id")

age_merge_table = conn.sql("SELECT SUM(AGE) FROM people_merge").df()
display(age_merge_table)

,sum(AGE)
0,266.0
